In [1]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

    Updating registry at `C:\Users\Prabhat Malhan\.julia\registries\General`
   Resolving package versions...
    Updating `C:\Users\Prabhat Malhan\.julia\environments\v1.6\Project.toml`
  [2913bbd2] + StatsBase v0.33.5
  No Changes to `C:\Users\Prabhat Malhan\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
   Installed Lathe ────────────────── v0.0.9
   Installed ClassImbalance ───────── v0.8.7
   Installed CommonSubexpressions ─── v0.3.0
   Installed StatsBase ────────────── v0.32.2
   Installed CategoricalArrays ────── v0.7.7
   Installed DataFrames ───────────── v0.20.2
   Installed PooledArrays ─────────── v0.5.3
   Installed PDMats ───────────────── v0.9.12
   Installed Tables ───────────────── v1.4.2
   Installed ArrayInterface ───────── v2.14.17
   Installed SpecialFunctions ─────── v0.10.3
   Installed FillArrays ───────────── v0.8.14
   Installed NLSolversBase ────────── v7.5.0
   Installed LineSearches ─────────── v7.1.1
   Installed UnPack ─────────

In [2]:
ENV["COLUMNS"]=1000

1000

In [3]:
df=DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [4]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [5]:
names(df)

14-element Vector{Symbol}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [6]:
countmap(df.Exited)

Dict{Int64, Int64} with 2 entries:
  0 => 7963
  1 => 2037

In [7]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df,Not([:RowNumber,:CustomerId,:Surname,:Geography,:Gender,:Male]))
first(df,5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


In [8]:
using Lathe.preprocess: TrainTestSplit
train,test=TrainTestSplit(df,.75);

In [11]:
fm=@formula(Exited~CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain)
logit = glm(fm,train,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.82151      0.166528     -10.94    <1e-27  -2.1479       -1.49512
CreditScore      -0.000491447  0.000182962   -2.69    0.0072  -0.000850046  -0.000132847
Age               0.0413272    0.00167772    24.63    <1e-99   0.0380389     0.0446154
Tenure           -0.00848962   0.00612098    -1.39    0.1655  -0.0204865     0.00350728
Balance         

In [12]:
prediction=predict(logit,test)

2504-element Vector{Union{Missing, Float64}}:
 0.22248472812418296
 0.13024818266599003
 0.029372571504503685
 0.12060729784918389
 0.10169686837629498
 0.05210380546861741
 0.084223272098675
 0.0335107332066154
 0.24070971596704696
 0.3631876784920079
 0.43346776208164906
 0.04066231913210156
 0.20172252531969603
 ⋮
 0.2721766329604983
 0.23038416572167852
 0.07249935115851036
 0.4361124709153253
 0.3042988117335077
 0.25986762346545483
 0.4915412102995943
 0.13849681395797198
 0.5341600414243155
 0.01751864615942
 0.20607714536836616
 0.1127900268828457